# Churn Modeling for Customer Retention

**Part 4 of 4 — Retail Customer Intelligence**  
Predict who is likely to churn. Uses **train-only features** from Notebook 1 and churn labels from the holdout window.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)


## Load Train Features and Churn Labels

In [ ]:
DATA_DIR = Path("..") / "data" / "processed"
MODELS_DIR = Path("..") / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

features = pd.read_csv(DATA_DIR / "customer_features_train.csv", parse_dates=["last_purchase"])
labels = pd.read_csv(DATA_DIR / "customer_churn_labels.csv", parse_dates=["snapshot_date", "train_end"])

churn_df = features.merge(labels[["customer_id", "churned"]], on="customer_id", how="left")
churn_df["churned"] = churn_df["churned"].fillna(0).astype(int)

churn_df.head()


## Basic EDA

In [ ]:
churn_rate = churn_df["churned"].mean()
print(f"Churn rate: {churn_rate:.2%}")

plt.figure(figsize=(4, 3))
churn_df["churned"].value_counts().plot(kind="bar")
plt.title("Churn Distribution")
plt.xlabel("Churned (1=yes)")
plt.ylabel("Count")
plt.show()


## Train/Validation Split (Time-Safe)

We split by last purchase date to avoid leakage.

In [ ]:
# Sort by last purchase and split chronologically
churn_df = churn_df.sort_values("last_purchase")
split_idx = int(len(churn_df) * 0.8)

train_df = churn_df.iloc[:split_idx].copy()
val_df = churn_df.iloc[split_idx:].copy()

X_train = train_df.drop(columns=["customer_id", "churned", "last_purchase"])
y_train = train_df["churned"]

X_val = val_df.drop(columns=["customer_id", "churned", "last_purchase"])
y_val = val_df["churned"]

X_train.head()


## Baseline Model: Logistic Regression

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

num_cols = X_train.select_dtypes(include=["number"]).columns

log_reg = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000, class_weight="balanced"))
])

log_reg.fit(X_train[num_cols], y_train)

val_probs_lr = log_reg.predict_proba(X_val[num_cols])[:, 1]
val_pred_lr = (val_probs_lr >= 0.5).astype(int)

print("LogReg ROC-AUC:", roc_auc_score(y_val, val_probs_lr))
print(classification_report(y_val, val_pred_lr))


## Tree Model: Random Forest (Nonlinear Baseline)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    class_weight="balanced_subsample",
    n_jobs=-1,
)

rf.fit(X_train[num_cols], y_train)
val_probs_rf = rf.predict_proba(X_val[num_cols])[:, 1]
val_pred_rf = (val_probs_rf >= 0.5).astype(int)

print("RF ROC-AUC:", roc_auc_score(y_val, val_probs_rf))
print(classification_report(y_val, val_pred_rf))


## Model Comparison

In [ ]:
rows = [
    ("LogisticRegression", roc_auc_score(y_val, val_probs_lr)),
    ("RandomForest", roc_auc_score(y_val, val_probs_rf)),
]

if 'val_probs_xgb' in globals():
    rows.append(("XGBoost", roc_auc_score(y_val, val_probs_xgb)))

if 'val_probs_lgbm' in globals():
    rows.append(("LightGBM", roc_auc_score(y_val, val_probs_lgbm)))

results = pd.DataFrame(rows, columns=["model", "roc_auc"]).sort_values("roc_auc", ascending=False)
results


## Calibration and Threshold Tuning

We calibrate probabilities and select thresholds based on business cost/benefit.

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import brier_score_loss

# Calibrate the RF model
cal_rf = CalibratedClassifierCV(rf, method="isotonic", cv=3)
cal_rf.fit(X_train[num_cols], y_train)

val_probs_rf_cal = cal_rf.predict_proba(X_val[num_cols])[:, 1]

print("RF Brier (uncal):", brier_score_loss(y_val, val_probs_rf))
print("RF Brier (cal):", brier_score_loss(y_val, val_probs_rf_cal))


In [ ]:
# Threshold selection using simple profit model
# Example: outreach cost = 1, expected saved revenue = 10 if we correctly catch a churner
COST = 1.0
BENEFIT = 10.0

thresholds = np.linspace(0.05, 0.95, 19)
profits = []

for t in thresholds:
    preds = (val_probs_rf_cal >= t).astype(int)
    tp = ((preds == 1) & (y_val == 1)).sum()
    fp = ((preds == 1) & (y_val == 0)).sum()
    profit = tp * BENEFIT - fp * COST
    profits.append(profit)

best_idx = int(np.argmax(profits))
best_t = thresholds[best_idx]

pd.DataFrame({"threshold": thresholds, "profit": profits}).sort_values("profit", ascending=False).head()


## Precision-Recall Curve and F-score Threshold

In [ ]:
from sklearn.metrics import precision_recall_curve

precision, recall, thresholds = precision_recall_curve(y_val, val_probs_rf_cal)

# Avoid division by zero
f1 = 2 * (precision * recall) / (precision + recall + 1e-9)

best_idx = int(np.argmax(f1))
best_threshold = thresholds[best_idx] if best_idx < len(thresholds) else 0.5

print("Best F1:", f1[best_idx])
print("Best threshold:", best_threshold)

plt.figure(figsize=(5, 4))
plt.plot(recall, precision)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve (Calibrated RF)")
plt.show()


## Gradient Boosting Models (XGBoost / LightGBM)

In [ ]:
# XGBoost (if installed)
try:
    from xgboost import XGBClassifier

    xgb = XGBClassifier(
        n_estimators=400,
        max_depth=5,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="auc",
        random_state=42,
    )

    xgb.fit(X_train[num_cols], y_train)
    val_probs_xgb = xgb.predict_proba(X_val[num_cols])[:, 1]
    print("XGBoost ROC-AUC:", roc_auc_score(y_val, val_probs_xgb))

except Exception as e:
    print("XGBoost not available:", e)


In [ ]:
# LightGBM (if installed)
try:
    from lightgbm import LGBMClassifier

    lgbm = LGBMClassifier(
        n_estimators=400,
        max_depth=-1,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
    )

    lgbm.fit(X_train[num_cols], y_train)
    val_probs_lgbm = lgbm.predict_proba(X_val[num_cols])[:, 1]
    print("LightGBM ROC-AUC:", roc_auc_score(y_val, val_probs_lgbm))

except Exception as e:
    print("LightGBM not available:", e)


## Feature Importance

In [ ]:
# Logistic coefficients
coef = pd.Series(log_reg.named_steps["clf"].coef_[0], index=num_cols)
coef.sort_values().plot(kind="barh", figsize=(6, 4))
plt.title("Logistic Regression Coefficients")
plt.show()

# Random forest feature importance
rf_importance = pd.Series(rf.feature_importances_, index=num_cols).sort_values(ascending=True)
rf_importance.plot(kind="barh", figsize=(6, 4))
plt.title("Random Forest Feature Importance")
plt.show()


## Threshold Selection (Top-N Targeting)

Select a percentile threshold for outreach based on predicted risk.

In [ ]:
# Example: target top 10% risk using best model (choose RF by default)
probs = val_probs_rf

threshold = np.quantile(probs, 0.90)
val_df = val_df.copy()
val_df["churn_risk"] = probs
val_df["target_flag"] = (val_df["churn_risk"] >= threshold).astype(int)

val_df[["customer_id", "churned", "churn_risk", "target_flag"]].head()


## Save Churn Risk for Dashboard

In [ ]:
# One row per customer: churn_risk for retention dashboard and targeting
churn_df["churn_risk"] = rf.predict_proba(
    churn_df.drop(columns=["customer_id", "churned", "last_purchase"])[num_cols]
)[:, 1]
churn_df[["customer_id", "churn_risk"]].to_csv(
    DATA_DIR / "customer_churn_risk.csv", index=False
)
print("Saved customer_churn_risk.csv for dashboard")


## Save Model Artifacts

In [ ]:
# Save calibrated RF if available
try:
    joblib.dump(cal_rf, MODELS_DIR / "churn_rf_calibrated.pkl")
except Exception:
    pass


In [ ]:
import joblib

joblib.dump(log_reg, MODELS_DIR / "churn_logreg.pkl")
joblib.dump(rf, MODELS_DIR / "churn_rf.pkl")
